# Caso Práctico: Llamada a funciones con GPT / ChatGPT para integración con servicios externos

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la integración del ChatBot implementando en el caso práctico anterior con el servicio de correo electrónico de Gmail.

La aplicación debe ser capaz de realizar diferentes funciones sobre los correos electrónicos del usuario. Por ejemplo, debe ser capaz de buscar todos los correos electrónicos de un remitente concreto y resumir su contenido.

Para ello, debe implementarse la integración nativa de funciones con GPT / ChatGPT (https://platform.openai.com/docs/guides/gpt/function-calling)

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
!pip install openai==0.28

## 1. Lectura de la API Key

In [ ]:
import openai

with open("/content/drive/MyDrive/api-keys/secret-key.txt") as f:
  openai.api_key = f.readline()

## 2. Integración con Gmail

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def obtener_correos(remitente):
    """Proporciona los correos electrónicos del usuario."""
    creds = None
    if os.path.exists('/content/drive/MyDrive/api-keys/token.json'):
        creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/api-keys/token.json', SCOPES)

    try:
        correos = """"""
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
        # Recorremos todos los correos electrónicos
        for msg in results['messages']:
          mensaje = service.users().messages().get(userId='me', id=msg['id']).execute()
          # Comparamos el remitente del correo
          if remitente in str(mensaje['payload']['headers'][14]):
            # Extraemos el contenido del correo
            datos = base64.b64decode(mensaje['payload']['parts'][0]['body']['data'], '-_')
            correos += f"""
'''
Contenido: {datos}
'''
"""
        return correos

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')

In [ ]:
print(obtener_correos("shramos@protonmail.com"))

## 3. Integración de la función con GPT / ChatGPT

Definimos las funciones que queremos proporcionarle a GPT para que identifique si es necesario invocarlas.

Referencia: https://platform.openai.com/docs/guides/gpt/function-calling

In [ ]:
functions = [
        {
            "name": "obtener_correos",
            "description": "Obtiene todos los correos electrónicos procedentes de un remitente concreto.",
            "parameters": {
                "type": "object",
                "properties": {
                    "remitente": {
                        "type": "string",
                        "description": "El correo electrónico del remitente, ej. santiago@gmail.com",
                    }
                },
                "required": ["remitente"],
            },
        }
    ]

In [ ]:
def obtener_completion(mensajes, model="gpt-3.5-turbo"):
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensajes,
      functions=functions, # Proporciono las funciones definidas previamente
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message # Retornamos el mensaje

## 4. Implementación de un ChatBot interactivo

In [ ]:
import json

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response_message = obtener_completion(context)

    # Comprobamos si GPT quiere invocar una funcion
    if response_message.get("function_call"):
        # Invocamos la funcion
        available_functions = {
            "obtener_correos": obtener_correos,
        }  # Podríamos tener más de una función

        # Obtenemos la funcion que quiere invocar GPT
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        # Obtenemos los argumentos de la funcion proporcionados por GPT
        function_args = json.loads(response_message["function_call"]["arguments"])
        # Invocamos la funcion
        function_response = function_to_call(remitente=function_args.get("remitente"))

        # Enviamos la respuesta de la función a GPT
        context.append(response_message)  # Respuesta del assistant
        context.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # Contenido de la función

        response_message = obtener_completion(context)

    context.append(response_message)
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response_message['content'], width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
def end_chat(event):
    panels.append(pn.pane.Alert("Chat terminado por el usuario.", alert_type='success'))
    context.append({'role': 'system', 'content':"Despídete del usuario de manera amable y amigable."})
    response_message = obtener_completion(context)
    context.append(response_message)
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response_message['content'], width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = []

context = [ {'role':'system', 'content':
"""
Eres un asistente virtual para gestionar y procesar el correo electrónico. \
Interactúa amablemente con el usuario y solicítale el correo electrónico de un \
remitente para obtener sus correos y empezar a trabajar sobre ellos.
"""} ]


inp = pn.widgets.TextInput(value="Hola", placeholder='Introduce texto aqui...')
button_conversation = pn.widgets.Button(name="Chat!")
button_end_chat = pn.widgets.Button(name="Terminar Chat")

button_end_chat.on_click(end_chat)

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat),
    pn.panel(interactive_conversation, loading_indicator=True, sizing_mode="stretch_both"),
)

dashboard